<a href="https://colab.research.google.com/github/HANE48/colab_Deep_learining/blob/main/Bert_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요한 라이브러리 설치

In [2]:
!pip install transformers torch
!pip install accelerate -U
!pip install transformers[torch]
!pip install datasets


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

전처리 데이터 준비

In [3]:
from datasets import load_dataset

dataset = load_dataset('glue', 'sst2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

파인 튜닝을 위한 모델 load

In [4]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

데이터 전처리

In [5]:
def encode(examples):
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)

dataset = dataset.map(encode, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

모델 학습

In [1]:
!pip install numpy>=1.24.1
!pip install pyarrow>=15.0.0

In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # 결과와 체크포인트 저장 경로
    num_train_epochs=2,              # 학습 에포크 수
    per_device_train_batch_size=8,   # 배치 크기
    warmup_steps=500,                # 워밍업을 위한 학습 단계 수
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로그 저장 경로
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train']
)

trainer.train()

Step,Training Loss
10,0.827500
20,0.700900
30,0.680200
40,0.746900
50,0.660200
60,0.696600
70,0.713600
80,0.681200
90,0.646300
100,0.633800


TrainOutput(global_step=16838, training_loss=0.22648366837998915, metrics={'train_runtime': 3606.4114, 'train_samples_per_second': 37.35, 'train_steps_per_second': 4.669, 'total_flos': 8860133233720320.0, 'train_loss': 0.22648366837998915, 'epoch': 2.0})

모델 평가

In [8]:
from datasets import load_dataset

dataset = load_dataset('glue', 'sst2')
def encode(examples):
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)

dataset = dataset.map(encode, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [11]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation']  # 검증 데이터셋 추가
)

In [12]:
results = trainer.evaluate(dataset['validation'])
print(results)

{'eval_loss': 0.3631761968135834, 'eval_runtime': 6.0826, 'eval_samples_per_second': 143.359, 'eval_steps_per_second': 17.92}
